In [1]:
# Demonstrates how to accessing data on a GCP bucket from a Colab notebook
# Adapted from version for AWS: https://github.com/Seiris21/2022_snowpack_capstone/blob/main/notebooks/data_ingestion/read-from-gcp.ipynb 

In [2]:
!pip install --quiet fsspec
!pip install --quiet gcsfs

In [3]:
#https://python.plainenglish.io/python-pull-gcp-cloud-storage-file-into-aws-s3-6f6cb8c144b2
# Install the google-cloud-bigquery and pydata-google-auth packages
# to use this code snippet.
import google.cloud.bigquery
import google.cloud.bigquery.magics
#import pydata_google_auth
#for google storage
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.oauth2 import service_account

In [4]:
import pandas as pd
import io, numpy as np, re, os, sys, json

In [5]:
# Add GCP credentials:
# 1. Create a service account under https://console.cloud.google.com/iam-admin/serviceaccounts/create ,
# assigning it the "Storage Legacy Bucket Owner" and "Storage Legacy Object Owner"
# roles
# (or another role/combination of roles that includes the needed permissions 
# like storage.buckets.get and storage.objects.get, 
# see  https://cloud.google.com/iam/docs/understanding-roles#cloud-storage-roles ).
# If one of the roles doesn't show up in the dropdown list, try
# finding it in the project's full roles list, via
# https://console.cloud.google.com/iam-admin/roles , 
# then select "Create role from this role", and then try to select the newly 
# created custom role in the dropdown list after restarting the service account
# creation.
# 2. Create and download a JSON key for this account via 
# https://console.cloud.google.com/iam-admin/serviceaccounts
# 3. Save the JSON key file in this notebook's folder and update the environment 
# variable as below

# Previously used:
# On Colab:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="generated-atlas-345415-685e2383eeaa.json"

In [6]:
#local variables for credentials
service_account_json = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
#Set your GCP project Name
BgProject="Snow Cast"
#Construct a BigQuery client object.
bq_client = bigquery.Client()
gcpbucket = storage.Client().get_bucket("snowcast_maindata")
#set the your S3 bucket name
s3bucket="my-aws-s3-bucket-name"
#Configure the magics to use your credentials and project.
google.cloud.bigquery.magics.context.project = BgProject
google.cloud.bigquery.magics.context.credentials = service_account_json
credentials = service_account.Credentials.from_service_account_file(service_account_json)
client = bigquery.Client(credentials= credentials,project=BgProject)

In [7]:
def load_gsdatafile(gcpbucket, filename):
    """
    this function returns a dataframe 
    from a gcp google storage file
    gcpbucket: gcp bucket name, filename: gcp file name
    """
    error='-1'
    try: 
        bucket=gcpbucket.name
        filename = filename
        gsuri = 'gs://{}/{}'.format(bucket, filename) 
        df = pd.read_csv(gsuri,  encoding='utf-8')
        df.head()
        return df
    except Exception as e:
        return error

In [8]:
#this function will load dataframe into S3 bucket
#df: dataframe, s3bucket: S3 bucket name, file_key: file name
def savefile_toS3(df, s3bucket, file_key):
    msg='-1'
    try: 
        bucket=s3bucket
        file_key = file_key
        s3uri = 's3://{}/{}'.format(bucket, file_key) 
        print(s3uri) 
        df.to_csv(s3uri)
        msg='Files has been saved successfully!'
        return msg
    except Exception as e:
        return msg

In [9]:
def load_gsdatafile_with_exception(gcpbucket, filename):
    
    bucket=gcpbucket.name
    filename = filename
    gsuri = 'gs://{}/{}'.format(bucket, filename) 
    df = pd.read_csv(gsuri,  encoding='utf-8')
    df.head()
    return df

In [10]:
# some tests:

In [11]:
%%time
filepath = "allregions_first.csv"
allregions_first_df=load_gsdatafile(gcpbucket,filepath)

CPU times: user 771 ms, sys: 128 ms, total: 899 ms
Wall time: 2.23 s


In [12]:
allregions_first_df.head()

,cell_id,date,SWE,region,geometry,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,...,datestring,centroid,center_lat,center_long,MOD10A1_SnowCover,MOD10A1_Albedo,MOD10A1_NDSI,MYD10A1_SnowCover,MYD10A1_Albedo,MYD10A1_NDSI
0,ASO_50M_SWE_USCARC_8,2017-07-17,48.561301,sierras,POLYGON ((-119.27063272296992 37.7439075502243...,4.697841,4.685963,1.381988,18.482857,0.573913,...,2017198,POINT (-13276514.66403197 3884236.855922926),37.739402,-119.264960,0.761684,0.367174,0.766883,0.814966,0.345533,0.818492
1,ASO_50M_SWE_USCARC_9,2017-07-17,33.924218,sierras,POLYGON ((-119.27035750488221 37.7349013266373...,4.736198,4.685963,1.381988,18.482857,0.573913,...,2017198,POINT (-13276484.103364434 3883445.3167475215),37.730396,-119.264686,0.666698,0.326389,0.669003,0.790095,0.339398,0.792235
2,ASO_50M_SWE_USCARC_26,2017-07-17,31.831418,sierras,POLYGON ((-119.25983912243692 37.7621381423023...,5.221427,4.685963,1.381988,18.482857,0.573913,...,2017198,POINT (-13275312.971020248 3885838.8167833057),37.757633,-119.254165,0.717794,0.387605,0.721743,0.660893,0.377931,0.665706
3,ASO_50M_SWE_USCARC_27,2017-07-17,41.723133,sierras,POLYGON ((-119.25956503213258 37.7531318793437...,5.322858,4.685963,1.381988,18.482857,0.573913,...,2017198,POINT (-13275282.53596653 3885047.4658173174),37.748627,-119.253892,0.758141,0.388365,0.761231,0.754647,0.370129,0.758559
4,ASO_50M_SWE_USCARC_28,2017-07-17,42.693821,sierras,POLYGON ((-119.2592910638845 37.74412560089217...,5.377803,4.685963,1.381988,18.482857,0.573913,...,2017198,POINT (-13275252.114465987 3884256.018790994),37.739620,-119.253619,0.732553,0.393211,0.737283,0.753537,0.346098,0.757798


In [13]:
%%time
filepath = "GRIDMET_Weather_Data.csv"
GRIDMET_df=load_gsdatafile(gcpbucket,filepath)

CPU times: user 19.8 s, sys: 4.05 s, total: 23.8 s
Wall time: 2min


In [14]:
GRIDMET_df.head()

,cell_id,geometry,date,precip_daily,wind_dir_avg,temp_min,temp_max,wind_vel
0,ASO_50M_SWE_USCAJW_3012,POLYGON ((-119.3966606691139 37.58805474778629...,2019-03-12,0.0,114.00,262.80,275.20,5.0
1,ASO_50M_SWE_USCAJW_3012,POLYGON ((-119.3966606691139 37.58805474778629...,2019-04-18,0.0,146.00,268.90,286.20,3.8
2,ASO_50M_SWE_USCAJW_3012,POLYGON ((-119.3966606691139 37.58805474778629...,2019-04-14,0.0,248.00,268.90,285.50,3.7
3,ASO_50M_SWE_USCAJW_3012,POLYGON ((-119.3966606691139 37.58805474778629...,2016-07-05,0.0,215.42,279.89,294.03,4.0
4,ASO_50M_SWE_USCAJW_3012,POLYGON ((-119.3966606691139 37.58805474778629...,2017-01-25,0.0,196.00,255.50,267.50,3.5


In [15]:
GRIDMET_df.describe()

,precip_daily,wind_dir_avg,temp_min,temp_max,wind_vel
count,5.377409e+06,5.377409e+06,5.377409e+06,5.377409e+06,5.377409e+06
mean,2.925739e+00,1.986025e+02,2.742573e+02,2.867352e+02,3.145493e+00
std,9.966575e+00,7.289872e+01,8.084030e+00,9.279131e+00,1.632548e+00
min,0.000000e+00,7.580000e+00,2.468000e+02,2.565000e+02,4.000000e-01
25%,0.000000e+00,1.500000e+02,2.688000e+02,2.801000e+02,2.100000e+00
50%,0.000000e+00,1.990000e+02,2.742900e+02,2.868000e+02,2.720000e+00
75%,6.000000e-01,2.479200e+02,2.800900e+02,2.931000e+02,3.700000e+00
max,1.333000e+02,3.520000e+02,2.986000e+02,3.153000e+02,1.660000e+01
